In [1]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset,DataLoader,TensorDataset
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import sklearn
from sklearn import metrics
import category_encoders as ce
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
#import data_loader_num_cat
import neural_net 
import sys
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
#from ray import tune
print(torch.__version__)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

1.8.1


In [2]:
sys.version

'3.6.13 |Anaconda, Inc.| (default, Feb 23 2021, 12:58:59) \n[GCC Clang 10.0.0 ]'

In [15]:
ray.shutdown()
ray.init(num_cpus=8, num_gpus=1,
        #object_store_memory=78643200
        )
assert ray.is_initialized() == True

2021-06-15 15:20:37,886	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


In [3]:
df = pd.read_csv('cars.csv')

# Convert boolean to zero/one as category type
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(int).astype(object)
        
# Use price as target and all other columns as features
target = 'price_usd'
features = df.columns[df.columns!=target]        

In [4]:
df.dtypes

manufacturer_name     object
model_name            object
transmission          object
color                 object
odometer_value         int64
year_produced          int64
engine_fuel           object
engine_has_gas        object
engine_type           object
engine_capacity      float64
body_type             object
has_warranty          object
state                 object
drivetrain            object
price_usd            float64
is_exchangeable       object
location_region       object
number_of_photos       int64
up_counter             int64
feature_0             object
feature_1             object
feature_2             object
feature_3             object
feature_4             object
feature_5             object
feature_6             object
feature_7             object
feature_8             object
feature_9             object
duration_listed        int64
dtype: object

In [5]:
# Drop na and define train-test set
df = df.dropna().copy()

target = 'price_usd'
features = df.drop(target, 1).columns

X_train, X_test, y_train, y_test = train_test_split(df[features], 
                                                    df[target],
                                                    test_size = 0.2, 
                                                    random_state = 2)

In [6]:
#CatBoost encoder used for features
encoder = ce.CatBoostEncoder()

for col in X_train.columns:
    if col in features:
        if X_train[col].dtype == object:
            X_train[col] = encoder.fit_transform(X_train[col], y_train)
            X_test[col] = encoder.transform(X_test[col])

In [7]:
def pd_to_ptrch(x):
    '''
    Makes a tensor from a
    pandas data frame.
    '''
    x_torch = torch.tensor(x.values)
    return x_torch

In [8]:
# creating tensor from targets_df 
X_train_torch = pd_to_ptrch(X_train)
y_train_torch = pd_to_ptrch(y_train).to(torch.float32)
y_train_torch = y_train_torch.view(-1,1)

X_test_torch = pd_to_ptrch(X_test)
y_test_torch = pd_to_ptrch(y_test)
y_test_torch = pd_to_ptrch(y_test).to(torch.float32)
y_test_torch = y_test_torch.view(-1,1)

data_train = TensorDataset(X_train_torch, y_train_torch)
data_test = TensorDataset(X_test_torch, y_test_torch)

In [9]:
#Initialize model:
net = neural_net.TheModel

In [10]:
#activate devices
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
def train_car(setting):
    training = torch.utils.data.DataLoader(data_train, 
                                           batch_size=int(setting["batch_size"]), 
                                           shuffle=False)
    
    testing = torch.utils.data.DataLoader(data_test, 
                                          batch_size=X_test.shape[0], 
                                          shuffle=False)
    model = net(
              torch.nn.ReLU, 
              #torch.nn.RReLU,
              #torch.nn.ReLU6,
              #torch.nn.LeakyReLU,
               #torch.nn.ELU,
                False,
              [len(X_train.columns), 500, 250, 1])
    
    optimizer = torch.optim.Adam(model.parameters(),lr=setting["lr"])
    loss_function = torch.nn.MSELoss()
    
    for i in range(50):
        for inputs, outputs in training:
            inputs, outputs = inputs.to(device), outputs.to(device)
            optimizer.zero_grad()
            results = model(inputs.float())
            loss = loss_function(results, outputs)
            loss.backward()
            optimizer.step()

        for inputs, outputs in testing:
            predicted = model(inputs.float()).detach().numpy()
            actual = outputs.numpy()
            acc = sklearn.metrics.r2_score(actual, predicted)
            #print(sklearn.metrics.r2_score(actual, predicted))
            #print(sklearn.metrics.mean_absolute_error(actual, predicted))
            #loss_list.append((sklearn.metrics.mean_absolute_error(actual, predicted),
             #                str(relu)))
            tune.report(loss=acc)
 
scheduler = ASHAScheduler(
                            metric="loss",
                            mode="max",
                            max_t=5, # defines max number of epochs
                            grace_period=1,
                            reduction_factor=2)
    
reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss"])

analysis = tune.run(
    train_car, config={"lr": tune.grid_search([0.001, 0.01, 0.1]),
                      "batch_size": tune.choice([64, 128, 256])}
                       #,resources_per_trial={'gpu': 1}
                       ,scheduler=scheduler
                       ,progress_reporter=reporter
                       ,num_samples=5
                        )

best_trial = analysis.get_best_trial("loss", "max", "last")
print('Best settings are: {}'.format(best_trial.config))  
print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))

2021-06-15 15:34:31,021	WARNING tune.py:494 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
2021-06-15 15:34:32,308	WARNING worker.py:1114 -- Warning: The actor ImplicitFunc has size 18343691 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


== Status ==
Memory usage on this node: 14.4/16.0 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/8 CPUs, 0/1 GPUs, 0.0/8.59 GiB heap, 0.0/4.29 GiB objects
Result logdir: /Users/janrathfelder/ray_results/train_car_2021-06-15_15-34-30
Number of trials: 15/15 (14 PENDING, 1 RUNNING)
+-----------------------+----------+-------+--------------+-------+
| Trial name            | status   | loc   |   batch_size |    lr |
|-----------------------+----------+-------+--------------+-------|
| train_car_6a009_00000 | RUNNING  |       |          128 | 0.001 |
| train_car_6a009_00001 | PENDING  |       |          128 | 0.01  |
| train_car_6a009_00002 | PENDING  |       |          256 | 0.1   |
| train_car_6a009_00003 | PENDING  |       |          256 | 0.001 |
| train_car_6a009_00004 | PENDING  |       |          256 | 0.01  |
| train_car_6a009_00005 | PENDING  |       |          128 | 0.1   |
| train_car_6a009_00006 |

Result for train_car_6a009_00000:
  date: 2021-06-15_15-34-42
  done: false
  experiment_id: f2e9d8d20d444a3ca52b53387375a97b
  hostname: Jans-MBP
  iterations_since_restore: 3
  loss: 0.7486532999564159
  node_ip: 192.168.1.112
  pid: 22698
  time_since_restore: 7.747696876525879
  time_this_iter_s: 2.55949068069458
  time_total_s: 7.747696876525879
  timestamp: 1623764082
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: 6a009_00000
  
== Status ==
Memory usage on this node: 14.5/16.0 GiB: ***LOW MEMORY*** less than 10% of the memory on this node is available for use. This can cause unexpected crashes. Consider reducing the memory used by your application or reducing the Ray object store size by setting `object_store_memory` when calling `ray.init`.
Using AsyncHyperBand: num_stopped=6
Bracket: Iter 4.000: None | Iter 2.000: 0.7387605655816099 | Iter 1.000: 0.7060550245041923
Resources requested: 8.0/8 CPUs, 0/1 GPUs, 0.0/8.59 GiB heap, 0.0/4.29 GiB objects
Result logdi

Result for train_car_6a009_00009:
  date: 2021-06-15_15-34-48
  done: true
  experiment_id: f6270d76dd3b4238bee81a1ddd0dc891
  hostname: Jans-MBP
  iterations_since_restore: 4
  loss: 0.7555332146610466
  node_ip: 192.168.1.112
  pid: 22781
  time_since_restore: 7.111100912094116
  time_this_iter_s: 1.318544864654541
  time_total_s: 7.111100912094116
  timestamp: 1623764088
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: 6a009_00009
  


2021-06-15 15:34:48,791	INFO tune.py:549 -- Total run time: 17.89 seconds (17.64 seconds for the tuning loop).


Result for train_car_6a009_00006:
  date: 2021-06-15_15-34-48
  done: true
  experiment_id: 623884a4e31840348d54750c0f237fdb
  hostname: Jans-MBP
  iterations_since_restore: 5
  loss: 0.7644374707188546
  node_ip: 192.168.1.112
  pid: 22726
  time_since_restore: 13.021300077438354
  time_this_iter_s: 1.639185905456543
  time_total_s: 13.021300077438354
  timestamp: 1623764088
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: 6a009_00006
  
== Status ==
Memory usage on this node: 14.3/16.0 GiB
Using AsyncHyperBand: num_stopped=15
Bracket: Iter 4.000: 0.7555332146610466 | Iter 2.000: 0.7387605655816099 | Iter 1.000: 0.7012973388450223
Resources requested: 0/8 CPUs, 0/1 GPUs, 0.0/8.59 GiB heap, 0.0/4.29 GiB objects
Result logdir: /Users/janrathfelder/ray_results/train_car_2021-06-15_15-34-30
Number of trials: 15/15 (15 TERMINATED)
+-----------------------+------------+-------+--------------+-------+-----------+
| Trial name            | status     | loc   |   batch_size |  

In [30]:
# Use optimized settings:
training = torch.utils.data.DataLoader(data_train, 
                                        batch_size=64, 
                                        shuffle=False)
    
testing = torch.utils.data.DataLoader(data_test, 
                                      batch_size=X_test.shape[0], 
                                      shuffle=False)
model = net(
              torch.nn.ReLU, 
              #torch.nn.RReLU,
              #torch.nn.ReLU6,
              #torch.nn.LeakyReLU,
               #torch.nn.ELU,
                False,
              [len(X_train.columns), 500, 250, 1])
    
optimizer = torch.optim.Adam(model.parameters(),lr=.001)
loss_function = torch.nn.MSELoss()
    
for epoch in range(25):
    for inputs, outputs in training:
        inputs, outputs = inputs.to(device), outputs.to(device)
        optimizer.zero_grad()
        results = model(inputs.float())
        loss = loss_function(results, outputs)
        loss.backward()
        optimizer.step()
    print('Loss: {}'.format(loss))

for inputs, outputs in testing:
    predicted = model(inputs.float()).detach().numpy()
    actual = outputs.numpy()
    acc = sklearn.metrics.r2_score(actual, predicted)
    print(sklearn.metrics.r2_score(actual, predicted))
    print(sklearn.metrics.mean_squared_error(actual, predicted))
    print(sklearn.metrics.mean_absolute_error(actual, predicted))


Loss: 6905502.0
Loss: 6604183.0
Loss: 9150695.0
Loss: 8590722.0
Loss: 8477340.0
Loss: 8034146.0
Loss: 8211848.0
Loss: 7952275.0
Loss: 7385473.0
Loss: 7192419.0
Loss: 6889360.5
Loss: 6926029.5
Loss: 6979483.0
Loss: 7490517.5
Loss: 7753079.0
Loss: 8025590.5
Loss: 7841205.5
Loss: 7898478.5
Loss: 7634609.5
Loss: 7670920.0
Loss: 7642657.0
Loss: 7247039.0
Loss: 7350573.0
Loss: 7466235.5
Loss: 6655489.0
0.7878543177864501
9110427.0
1920.7031
